In [6]:
# ==============================================
# 01_extract_clinical.ipynb
# Purpose: Extract and clean TCGA-COAD/READ clinical data for all cases.
# ==============================================

import os
import pandas as pd
import glob

# Set working directory to project root
os.chdir(r"C:\Users\Negar\Desktop\paper_results\Myself\cr_coad_project")

# Define paths
raw_clinical_dir = "data/raw/clinical"
out_dir = "data/raw/tcga"
os.makedirs(out_dir, exist_ok=True)


In [7]:
# ------------------------------------------------------------
# Step 1: Find all available TSV files
# ------------------------------------------------------------
files = glob.glob(os.path.join(raw_clinical_dir, "*.tsv"))
if not files:
    raise SystemExit(f"❌ No clinical TSVs found in {raw_clinical_dir}")
print(f"Found {len(files)} clinical files")


Found 10 clinical files


In [8]:
# ------------------------------------------------------------
# Step 2: Load & standardize each TSV
# ------------------------------------------------------------
def load_tsv(path):
    df = pd.read_csv(path, sep="\t", dtype=str)
    df.columns = df.columns.str.strip()
    return df

dfs = []
for f in files:
    df = load_tsv(f)
    if "cases.submitter_id" in df.columns:
        df = df.rename(columns={"cases.submitter_id": "submitter_id"})
    if "cases.case_id" in df.columns:
        df = df.rename(columns={"cases.case_id": "case_id"})
    dfs.append(df)

if not dfs:
    raise SystemExit("❌ No clinical-style TSVs with case identifiers found.")

In [9]:
# ------------------------------------------------------------
# Step 3: Combine and flatten
# ------------------------------------------------------------
big = pd.concat(dfs, axis=0, ignore_index=True, sort=False)

# Ensure we have identifiers
if "case_id" not in big.columns and "submitter_id" not in big.columns:
    raise SystemExit("❌ No case_id or submitter_id found in any TSV.")

# Prefer case_id, fallback to submitter_id
id_col = "case_id" if "case_id" in big.columns else "submitter_id"

# One row per patient — keep first non-null value per column
big = big.groupby(id_col).agg(lambda s: s.dropna().iloc[0] if s.dropna().shape[0] > 0 else pd.NA).reset_index()

In [10]:
# ------------------------------------------------------------
# Step 4: Derive metastasis_status
# ------------------------------------------------------------
def derive_m_status(row):
    # Check AJCC M categories
    for col in [
        "diagnoses.ajcc_pathologic_m",
        "diagnoses.ajcc_clinical_m",
        "diagnoses.ajcc_m_pathologic",
        "diagnoses.ajcc_m",
    ]:
        if col in row and pd.notna(row[col]):
            v = str(row[col]).upper()
            if "M1" in v or v.strip() in ["1", "M1.0"]:
                return 1
            if "M0" in v or v.strip() in ["0"]:
                return 0
    # Check stage info
    for col in [
        "diagnoses.ajcc_pathologic_stage",
        "diagnoses.ajcc_clinical_stage",
        "diagnoses.uicc_pathologic_stage",
    ]:
        if col in row and pd.notna(row[col]):
            v = str(row[col]).upper()
            if "IV" in v or "4" in v:
                return 1
            if any(x in v for x in ["I", "II", "III", "1", "2", "3"]):
                return 0
    # Check metastasis_at_diagnosis
    if "diagnoses.metastasis_at_diagnosis" in row and pd.notna(row["diagnoses.metastasis_at_diagnosis"]):
        v = str(row["diagnoses.metastasis_at_diagnosis"]).lower()
        if any(x in v for x in ["yes", "present", "1"]):
            return 1
        if any(x in v for x in ["no", "absent", "0"]):
            return 0
    return pd.NA

big["metastasis_status"] = big.apply(derive_m_status, axis=1)


In [11]:
# ------------------------------------------------------------
# Step 5: Build a clean manifest
# ------------------------------------------------------------
def get_col(preferred_cols):
    for c in preferred_cols:
        if c in big.columns:
            return big[c]
    return pd.NA

manifest_df = pd.DataFrame({
    "case_id": big[id_col],
    "submitter_id": get_col(["submitter_id"]),
    "primary_diagnosis": get_col(["diagnoses.primary_diagnosis", "diagnoses.primary_disease"]),
    "gender": get_col(["demographic.gender", "demographic.sex"]),
    "age_at_diagnosis": get_col(["diagnoses.age_at_diagnosis", "demographic.age_at_index"]),
    "tumor_stage": get_col(["diagnoses.ajcc_pathologic_stage", "diagnoses.ajcc_clinical_stage"]),
    "metastasis_status": big["metastasis_status"],
    "vital_status": get_col(["demographic.vital_status"]),
    "days_to_death": get_col(["demographic.days_to_death", "diagnoses.days_to_death"]),
})


In [12]:
# ------------------------------------------------------------
# Step 6: Save clean outputs (overwrite allowed)
# ------------------------------------------------------------
full_csv = os.path.join(out_dir, "tcga_coad_read_clinical.csv")
manifest_csv = os.path.join(out_dir, "tcga_clinical_manifest.csv")

# Always overwrite (safe reset)
for f in [full_csv, manifest_csv]:
    if os.path.exists(f):
        os.remove(f)

big.to_csv(full_csv, index=False)
manifest_df.to_csv(manifest_csv, index=False)

print(f"✅ Saved full data to: {full_csv}")
print(f"✅ Saved manifest summary to: {manifest_csv}")


✅ Saved full data to: data/raw/tcga\tcga_coad_read_clinical.csv
✅ Saved manifest summary to: data/raw/tcga\tcga_clinical_manifest.csv


In [13]:
# ------------------------------------------------------------
# Step 7: Quick sanity check
# ------------------------------------------------------------
print("\n📊 Value counts for metastasis_status:")
print(manifest_df["metastasis_status"].value_counts(dropna=False))

print("\n✅ Total patients:", manifest_df.shape[0])
print("✅ Patients with metastasis labels:", manifest_df['metastasis_status'].notna().sum())


📊 Value counts for metastasis_status:
metastasis_status
0       456
<NA>    111
1        66
Name: count, dtype: int64

✅ Total patients: 633
✅ Patients with metastasis labels: 522
